In [ ]:
# %pip install newsapi-python

In [ ]:
from newsapi import NewsApiClient
from langchain.docstore.document import Document
from langchain.chains import create_qa_with_sources_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

## NewsAPI

In [8]:
today = date.today()
last_week = today - timedelta(days=7)

latest_news = newsapi.get_everything(
    q='artificial intelligence',
    from_param=last_week.strftime("%Y-%m-%d"),
    to=today.strftime("%Y-%m-%d"),
    sort_by='relevancy',
    language='en'
)

In [9]:
latest_news['articles'][:5]

[{'source': {'id': None, 'name': 'CNET'},
  'author': 'Gael Cooper',
  'title': "A New 'Trustmark' Could Show Music Uses AI Responsibly",
  'description': 'AI:OK, co-founded by musician Martin Clancy, seeks to set standards involving artificial intelligence and the music industry.',
  'url': 'https://www.cnet.com/tech/services-and-software/a-new-trustmark-could-show-music-uses-ai-responsibly/',
  'urlToImage': 'https://www.cnet.com/a/img/resize/f3f37277dcf1af0ded788ada2953db18a1098838/hub/2024/06/26/c5c0ed80-f053-4a74-a14c-a556f6386567/neon-woman-ai.jpg?auto=webp&fit=crop&height=675&width=1200',
  'publishedAt': '2024-06-27T13:00:09Z',
  'content': 'Use of artificial intelligence in the music industry has been a hot topic, and there are no universal standards regarding the melding of the technology and the art form. So on Thursday, two music-ind… [+1995 chars]'},
 {'source': {'id': None, 'name': 'NPR'},
  'author': 'The Associated Press',
  'title': 'Amazon becomes the 5th U.S. company

In [10]:

docs = [
    Document(
    page_content=article['title'] + '\n\n' + article['description'], 
    metadata={
        'source': article['url'],
    }
    ) for article in latest_news['articles']
]

print(docs[0].page_content)
print(docs[0].metadata)

A New 'Trustmark' Could Show Music Uses AI Responsibly

AI:OK, co-founded by musician Martin Clancy, seeks to set standards involving artificial intelligence and the music industry.
{'source': 'https://www.cnet.com/tech/services-and-software/a-new-trustmark-could-show-music-uses-ai-responsibly/'}


In [13]:
llm = ChatOpenAI()
embeddings = OpenAIEmbeddings(show_progress_bar=True)

qa_chain = create_qa_with_sources_chain(llm)

doc_prompt = PromptTemplate(
    template="Content: {page_content}\nSource: {source}",
    input_variables=["page_content", "source"],
)

final_qa_chain = StuffDocumentsChain(
    llm_chain=qa_chain,
    document_variable_name="context",
    document_prompt=doc_prompt,
)

index = FAISS.from_documents(docs, embedding=embeddings)


chain = RetrievalQA(
    retriever=index.as_retriever(), 
    combine_documents_chain=final_qa_chain
)

C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.30it/s]
C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `RetrievalQA` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_retrieval_chain instead.
  warn_deprecated(


In [14]:
question = """
What is the most important news about artificial intelligence from last week?
"""

answer = chain.run(question)

print(answer)

C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.13it/s]


{
    "answer": "The most important news about artificial intelligence from last week was the Caribbean nation capitalizing on the AI boom. This news highlights the significant advancements and opportunities AI presents for the future.",
    "sources": [
        "https://www.foxnews.com/tech/fox-news-ai-newsletter-caribbean-nation-capitalizes-ai-boom"
    ]
}
